In [15]:
import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2

print('Tensorflow version {}'.format(tf.__version__))

Tensorflow version 1.12.0


In [27]:
mnv2 = mobilenet_v2.MobileNetV2(
    input_shape=(224, 224, 3),
    alpha=1.4,
    weights='imagenet',
    pooling='avg'
)

mnv2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 48) 1296        Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 48) 192         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [33]:
# Trainable Layers and Model Truncation
# Source: https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/
trainable_starting_with = 'block_16'
N_truncated_layers = 1
interesting_keys = ['trainable', 'filters']
trainable_layer_reached = False

# Truncate Layers
"""
from keras.models import Model
from keras.layers import Dense, Input

# Assume we have a pretrained model
net_input = Input(shape=(10))
net = Dense(4) (net_input)
net_output = Dense(1) (net)
model1 = Model(net_input, net_output)

# Get input
new_input = model1.input
# Find the layer to connect
hidden_layer = model1.layers[-2].output
# Connect a new layer on it
new_output = Dense(2) (hidden_layer)
# Build a new model
model2 = Model(new_input, new_output)
"""
# Source: https://github.com/keras-team/keras/issues/2371
for i in range(N_truncated_layers):
    print("POP!")
    mnv2.layers.pop()


# Set Trainable Layers
for layer in mnv2.layers:
    # Only make certain layers trainable
    config = layer.get_config()
    if trainable_starting_with in config['name']:
        trainable_layer_reached = True
    if trainable_layer_reached:
        layer.trainable = True
    else:
        layer.trainable = False
    print('{}'.format(config['name']))
    if layer.trainable:
        print('\tInput Shape: {}'.format(layer.input_shape))
        for key in config.keys():
            if key in interesting_keys:
                print('\t{}: {}'.format(key, config[key]))
        print('\tOutput Shape: {}'.format(layer.output_shape))

POP!
input_6
Conv1_pad
Conv1
bn_Conv1
Conv1_relu
expanded_conv_depthwise
expanded_conv_depthwise_BN
expanded_conv_depthwise_relu
expanded_conv_project
expanded_conv_project_BN
block_1_expand
block_1_expand_BN
block_1_expand_relu
block_1_pad
block_1_depthwise
block_1_depthwise_BN
block_1_depthwise_relu
block_1_project
block_1_project_BN
block_2_expand
block_2_expand_BN
block_2_expand_relu
block_2_depthwise
block_2_depthwise_BN
block_2_depthwise_relu
block_2_project
block_2_project_BN
block_2_add
block_3_expand
block_3_expand_BN
block_3_expand_relu
block_3_pad
block_3_depthwise
block_3_depthwise_BN
block_3_depthwise_relu
block_3_project
block_3_project_BN
block_4_expand
block_4_expand_BN
block_4_expand_relu
block_4_depthwise
block_4_depthwise_BN
block_4_depthwise_relu
block_4_project
block_4_project_BN
block_4_add
block_5_expand
block_5_expand_BN
block_5_expand_relu
block_5_depthwise
block_5_depthwise_BN
block_5_depthwise_relu
block_5_project
block_5_project_BN
block_5_add
block_6_expand

In [21]:
# Feed data into the network
# Source: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html